<a href="https://colab.research.google.com/github/YunjiSon/2022spring/blob/main/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 7.7 MB/s 
     |████████████████████████████████| 2.0 MB 43.6 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 47.6 MB/s 
     |████████████████████████████████| 256 kB 45.4 MB/s 
     |████████████████████████████████| 271 kB 45.3 MB/s 
     |████████████████████████████████| 94 kB 677 kB/s 
     |████████████████████████████████| 144 kB 59.4 MB/s 
     |████████████████████████████████| 11.1 MB 32.7 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 80 kB 11.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 856 

In [ ]:
gr.Interface.load("google/vit-base-patch16-224").launch()
#이렇게 한줄만에 끝나는 애들도 있음.
#안됨 하나하나 다 해줘야하는듯

In [2]:
gr.Interface.load("huggingface/gpt2").launch()
#얜 됨

Fetching model from: https://huggingface.co/gpt2
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://44915.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f24dc1eb710>,
 'http://127.0.0.1:7860/',
 'https://44915.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


##image classification

In [4]:
!pip install transformers
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#pretrained는 데이터로부터 잘 학습이 된 행렬임. feature extraction을 할 수 있는 모델
#그 extract된 feature를 보고 image classification을 해내는 구나.


inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
#그 중에서 확률이 제일 큰거를 찾아라
print("Predicted class:", model.config.id2label[predicted_class_idx])
#총 1000개의 class가 이미 들어있는데 그중에 하나를 뽑아내는 것이다.
#즉, 이미저 넣으면 그거 classification 해주는 모델.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.7 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


프로그램 구현되는거 알았으면 이제 gradio화 해야함.

In [5]:
def func (image): #위의 이미지를 여기다가 받음. 
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

In [6]:
#gradio 화
gr.Interface(fn=func, inputs='image', outputs='text').launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17482.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f24d963f050>,
 'http://127.0.0.1:7861/',
 'https://17482.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


In [7]:
#예제 올려놓아서 확인하기 쉽게 하기 위해서
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")
#여기 콜랍에 다운받아짐.

0

In [8]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#입출력 유형에 함수로 적어주면 됨.
#tiger랑 dog list화 해서 적워둬야 함.

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://40521.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f23a7588b90>,
 'http://127.0.0.1:7862/',
 'https://40521.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


##Fill-mask

In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
#그 모델 이름을 이렇게 하고 거기서 밑에처럼 나오게 한다.
unmasker("Hello I'm a [MASK] model.")
#이렇게 나올 확률이 제일 높은 거부터 보여줌.
#결과가 지금 리스트로 딕셔너리가 5개들어와있음. 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

gradio로 만들기

In [11]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  #dataframe화 시키기. list를 데이터 프레임으로
  return df

In [12]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]
#각각에 잘 어울리는 것들을 높은 확률 순으로 ㅂ보여줌.

gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://16828.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f239f977bd0>,
 'http://127.0.0.1:7863/',
 'https://16828.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


##Token Classification
natural language를 understanding 하는. natural language있다면 (문장) Named Entity Recognition(NER-중요한, 특정한 것에 대한 명사를 recognize해주는 것. 중요 단체,위치, 사람, Miscellaneous)

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#두개의 pretrained model 가져옴

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)
#두개의 dictionary로 구성된 결과.

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


gradio 만들기

In [15]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df
  #즉, 리스트 안에 dictionary돼 있을 때 바로 데이터프레임화 할 수 있음.

In [16]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]


gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://44995.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f239c793910>,
 'http://127.0.0.1:7864/',
 'https://44995.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


##Sentence similarity
raw data를 embeddings로 바꿔준다. (벡터로) 쉽게 계산하기 위해서

In [17]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 16.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=32289ce33fadd87c68a856bfcced520fd1d1dd16c0eee13ce2c29eec2be26b98
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [18]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)
#list가 하나에 두개씩 들어가있음

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

[[ 1.88456848e-01  1.74256176e-01  5.44777215e-02  2.90517420e-01
   1.67664081e-01 -4.72069457e-02  6.45580173e-01  1.59808800e-01
   2.26892605e-01 -3.08905598e-02  2.55883545e-01 -5.25876768e-02
  -2.26101175e-01 -5.71063384e-02  1.30426243e-01  1.24953501e-01
   3.17496210e-01  1.94444105e-01 -5.86325705e-01 -1.25861000e-02
   6.09909177e-01  1.64327413e-01  3.33113708e-02 -2.73830682e-01
  -2.89757580e-01 -2.11196810e-01 -2.26138867e-02 -1.70359373e-01
   1.61589950e-01  6.08273745e-02 -2.41624042e-01  1.85792103e-01
   4.27409589e-01  1.92951798e-01 -7.23447800e-02  1.66111022e-01
   1.04428008e-01  2.04772323e-01  2.11167201e-01  1.99740008e-01
  -9.40827355e-02 -1.73836619e-01  6.42733201e-02  2.80254930e-01
  -2.95305759e-01  6.20952919e-02  1.04276791e-01 -2.36439407e-02
   1.29131526e-01 -1.26174763e-01 -1.78990051e-01  3.70057598e-02
  -6.12506211e-01  5.02983071e-02  1.77303478e-01  2.24941239e-01
   1.73860714e-01 -3.84028591e-02 -2.12868378e-01  2.58492470e-01
  -1.21016

In [19]:
len(embeddings)
#두개 있음.

2

In [20]:
len(embeddings[1])

384

In [21]:
len(embeddings[0])

384

즉, 384개의 숫자가 리스트로 있구나 이거 하나가 embedding이고 이게 feature vector이다. 이 두개를 가지고 similarity를 계산.

[1,2] [2,1]을 좌표평면에 찍어보고. 그것을 원점과 선을 긋고 두 선의 각도 차이로 유사도 차이를 알 수 있음. 세타가 0이면 완전 똑같은 것.
cos 0이 들어가면 1, cos90이면 2분의 파이라 0나옴. 이게 유사도의 결과.

>두 벡터의 유사도는 각도 값, 이걸 cos으로 계산해서 cosine similarity라고 함. 

In [22]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

tensor([[0.5726]])

gradio

In [23]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [25]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]


In [26]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://15826.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f239619af90>,
 'http://127.0.0.1:7865/',
 'https://15826.gradio.app')